In [28]:
import random
import os

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

# from catboost import CatBoostClassifier
# from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

# 변수고정

In [29]:
class CFG:
    user_seed = 42
    target = 'Prospect'

# Seed 고정

In [30]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED']=str(seed)
    np.random.seed(seed)
    
seed_everything(CFG.user_seed)

# Load data

In [31]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [32]:
train.isnull().sum().sum()

0

In [6]:
train.head()

,ID,Age,Height,Weight,Position,PreferredFoot,AttackingWorkRate,DefensiveWorkRate,PaceTotal,ShootingTotal,PassingTotal,DribblingTotal,DefendingTotal,PhysicalityTotal,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,STRating,LWRating,LFRating,CFRating,RFRating,RWRating,CAMRating,LMRating,CMRating,RMRating,LWBRating,CDMRating,RWBRating,LBRating,CBRating,RBRating,GKRating,Prospect
0,TRAIN_0000,21,182,82,CB,Right,Medium,Medium,4.6377,2.1053,3.2353,3.9394,6.3158,6.2295,4.8864,2.4176,6.7045,5.1807,1.8391,5.1087,2.7586,1.8889,5.3571,5.7303,4.8193,6.0976,5.0000,4.5312,5.3333,4.8684,6.8493,6.6667,6.4789,2.0690,5.7647,6.4773,4.8936,3.8095,4.5349,5.3012,6.7416,6.7816,6.6667,0.9091,0.7955,0.3297,0.7865,0.9091,4.3836,4.6053,4.4737,4.4737,4.4737,4.6053,4.5333,4.8649,4.9315,4.8649,6.1972,6.0000,6.1972,6.4286,6.5278,6.4286,0.875,0
1,TRAIN_0001,21,160,75,ST,Left,High,Medium,7.3913,7.3684,4.2647,5.9091,2.6316,6.2295,4.5455,7.6923,6.2500,5.4217,6.5517,7.3913,6.4368,5.2222,4.5238,5.8427,8.4337,7.1951,6.9737,6.4062,8.8000,6.9737,6.8493,7.4074,7.0423,7.5862,3.7647,3.7500,7.1277,6.4286,8.6047,6.1446,2.8090,3.3333,2.9762,1.2500,1.1364,0.6593,0.6742,1.3636,6.9863,6.8421,6.8421,6.8421,6.8421,6.8421,6.6667,6.4865,5.8904,6.4865,5.2113,4.4286,5.2113,5.0000,4.1667,5.0000,1.250,0
2,TRAIN_0002,18,165,60,RB,Right,Medium,Medium,6.2319,3.2895,3.9706,3.9394,4.8684,4.4262,5.9091,4.0659,5.2273,5.5422,2.1839,5.3261,3.3333,4.4444,5.4762,5.6180,6.7470,6.8293,6.1842,1.8750,6.1333,3.1579,6.1644,6.6667,4.2254,4.2529,4.0000,4.8864,5.6383,4.1667,4.0698,3.7349,5.0562,5.9770,5.9524,0.7955,0.9091,1.3187,1.0112,1.0227,4.5205,5.2632,4.7368,4.7368,4.7368,5.2632,4.9333,5.2703,4.9315,5.2703,5.7746,5.1429,5.7746,5.8571,5.0000,5.8571,0.750,0
3,TRAIN_0003,20,190,72,CB,Right,Medium,Medium,4.7826,1.0526,0.4412,0.3030,4.6053,4.0984,1.9318,1.6484,4.4318,2.6506,2.6437,2.0652,2.6437,2.3333,2.0238,2.8090,5.5422,5.7317,3.2895,2.6562,4.4000,2.3684,6.0274,5.5556,4.3662,2.2989,4.0000,5.5682,2.2340,2.1429,3.2558,3.2530,5.2809,5.0575,5.2381,1.2500,0.7955,0.8791,0.8989,1.3636,2.3288,2.3684,2.2368,2.2368,2.2368,2.3684,2.1333,2.2973,2.3288,2.2973,4.0845,3.7143,4.0845,4.5714,4.5833,4.5714,0.875,0
4,TRAIN_0004,21,181,77,ST,Right,High,Medium,8.2609,7.3684,5.4412,6.9697,1.3158,5.9016,6.8182,7.9121,6.2500,6.6265,7.2414,7.7174,4.8276,6.7778,5.1190,7.5281,9.1566,7.9268,8.4211,6.5625,6.6667,6.9737,3.0137,7.1605,6.3380,7.5862,5.4118,1.7045,7.9787,6.3095,6.8605,6.3855,2.5843,1.6092,1.0714,1.0227,0.6818,1.2088,1.3483,0.4545,7.6712,7.7632,7.6316,7.6316,7.6316,7.7632,7.4667,7.5676,6.3014,7.5676,5.2113,4.2857,5.2113,4.7143,3.3333,4.7143,1.125,0


In [7]:
test.head()

,ID,Age,Height,Weight,Position,PreferredFoot,AttackingWorkRate,DefensiveWorkRate,PaceTotal,ShootingTotal,PassingTotal,DribblingTotal,DefendingTotal,PhysicalityTotal,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,STRating,LWRating,LFRating,CFRating,RFRating,RWRating,CAMRating,LMRating,CMRating,RMRating,LWBRating,CDMRating,RWBRating,LBRating,CBRating,RBRating,GKRating
0,TEST_0000,17,184,75,GK,Right,Medium,Medium,3.7681,5.3947,4.8529,4.3939,0.7895,2.7869,0.7955,0.4396,0.7955,0.7229,0.2299,0.7609,0.6897,1.1111,2.3810,0.7865,1.3253,0.0000,1.0526,0.7812,3.8667,3.4211,5.8904,2.5926,2.3944,0.3448,1.0588,1.1364,0.3191,2.5000,1.2791,2.1687,0.5618,0.8046,0.9524,5.9091,6.2500,6.1538,5.0562,6.2500,0.0000,0.1316,0.1316,0.1316,0.1316,0.1316,0.2667,0.1351,0.5479,0.1351,0.2817,0.4286,0.2817,0.2857,0.2778,0.2857,5.500
1,TEST_0001,19,188,80,ST,Right,High,Medium,4.9275,4.8684,1.6176,3.1818,1.4474,5.2459,2.6136,6.0440,6.2500,3.8554,4.7126,4.5652,3.4483,2.8889,1.7857,5.0562,5.5422,5.9756,5.3947,2.8125,4.6667,4.7368,6.5753,4.6914,6.0563,4.5977,5.6471,1.7045,5.4255,4.1667,5.8140,4.2169,2.2472,2.1839,1.5476,0.6818,0.7955,0.6593,1.0112,1.0227,5.0685,4.6053,4.7368,4.7368,4.7368,4.6053,4.4000,4.1892,3.4247,4.1892,3.0986,2.7143,3.0986,3.0000,3.0556,3.0000,0.750
2,TEST_0002,17,173,68,CAM,Right,High,High,6.9565,6.3158,7.6471,8.3333,6.5789,6.0656,7.1591,7.1429,4.2045,8.7952,5.2874,8.6957,7.8161,4.7778,8.3333,8.6517,7.5904,7.3171,8.9474,7.6562,9.3333,5.0000,6.5753,8.0247,4.2254,7.2414,8.4706,8.0682,8.0851,8.4524,5.2326,7.3494,6.2921,7.5862,7.1429,0.6818,0.9091,0.9890,0.5618,1.2500,7.1233,8.4211,8.1579,8.1579,8.1579,8.4211,8.5333,8.3784,8.6301,8.3784,8.3099,8.2857,8.3099,8.1429,7.0833,8.1429,1.125
3,TEST_0003,20,178,69,CAM,Right,Medium,Medium,6.8116,6.4474,5.4412,6.8182,2.5000,5.4098,5.5682,7.3626,5.0000,6.6265,6.4368,7.7174,4.1379,6.3333,6.0714,7.5281,7.7108,6.9512,7.8947,6.4062,7.2000,5.2632,5.3425,6.6667,5.4930,6.2069,5.5294,4.2045,7.0213,7.2619,7.5581,6.6265,2.3596,3.2184,3.8095,1.0227,0.7955,1.0989,1.4607,1.3636,6.7123,7.3684,7.1053,7.1053,7.1053,7.3684,7.3333,7.1622,6.7123,7.1622,5.6338,5.1429,5.6338,5.2857,4.1667,5.2857,1.375
4,TEST_0004,19,176,72,CAM,Left,High,Low,5.5072,4.7368,4.5588,6.3636,3.1579,3.4426,4.7727,5.1648,4.0909,5.9036,4.8276,7.5000,5.8621,5.3333,6.3095,7.1910,6.5060,5.9756,6.8421,3.9062,6.9333,5.9211,5.0685,4.8148,3.5211,4.9425,4.5882,3.1818,4.4681,5.3571,5.4651,5.7831,4.7191,3.3333,4.5238,0.7955,1.2500,0.9890,0.3371,0.5682,5.2055,6.0526,5.7895,5.7895,5.7895,6.0526,6.0000,5.9459,5.4795,5.9459,5.0704,4.7143,5.0704,4.8571,4.0278,4.8571,0.750


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3019 entries, 0 to 3018
Data columns (total 66 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 3019 non-null   object 
 1   Age                3019 non-null   int64  
 2   Height             3019 non-null   int64  
 3   Weight             3019 non-null   int64  
 4   Position           3019 non-null   object 
 5   PreferredFoot      3019 non-null   object 
 6   AttackingWorkRate  3019 non-null   object 
 7   DefensiveWorkRate  3019 non-null   object 
 8   PaceTotal          3019 non-null   float64
 9   ShootingTotal      3019 non-null   float64
 10  PassingTotal       3019 non-null   float64
 11  DribblingTotal     3019 non-null   float64
 12  DefendingTotal     3019 non-null   float64
 13  PhysicalityTotal   3019 non-null   float64
 14  Crossing           3019 non-null   float64
 15  Finishing          3019 non-null   float64
 16  HeadingAccuracy    3019 

# Data preprocessing

## Label Encoding
- qualitative to quantitative

In [33]:
qual_col = ['Position', 'PreferredFoot', 'AttackingWorkRate', 'DefensiveWorkRate']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    
    test[i] = le.transform(test[i])
    
print('Done.')

Done.


# X, y split

In [34]:
X = train.drop(columns=['ID',CFG.target], axis=1)
y = train[CFG.target]

# Classification model fit

In [35]:
model = XGBClassifier(random_state=CFG.user_seed)

model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, ...)

In [36]:
X_test = test.drop('ID', axis=1)

pred = model.predict(X_test)

# Submit

In [37]:
submit = pd.read_csv('data/sample_submission.csv')
submit[CFG.target] = pred

In [38]:
submit

,ID,Prospect
0,TEST_0000,1
1,TEST_0001,0
2,TEST_0002,1
3,TEST_0003,0
4,TEST_0004,1
...,...,...
1621,TEST_1621,1
1622,TEST_1622,1
1623,TEST_1623,0
1624,TEST_1624,1


In [39]:
submit.to_csv('data/baseline_xgb.csv', index=False)